It triggers the webcam 
1>The background for the initial frame must be static so that Python can compare and detct changes

# check out thresholding ,dilation sophisticated kernels and gaussian blur

In [6]:
import cv2
from datetime import datetime
import pandas

# None  datatype is a special that allows you to create and assign nothing to it
first_frame=None
video=cv2.VideoCapture(0)
status_list=[None,None]
times=[]
df=pandas.DataFrame(columns=["Start","End"])

while True:
    check,frame=video.read()
    
    status=0
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #We apply gaussian blur to make gray image blurry so as to smooth the image as it in turn removes noise and increases accuracy    
    gray=cv2.GaussianBlur(gray,(21,21),0)
    
    if first_frame is None:
        first_frame=gray
        continue
        
    # We calculate the difference image between the current and the initial image
    delta_frame=cv2.absdiff(first_frame,gray)
    
    # We calculate the threshold image
    thresh_delta=cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1]
    
    # We dilate the thresholded image
    thresh_dilate=cv2.dilate(thresh_delta,None,iterations=15)
    
    # 
    (cnts,_)=cv2.findContours(thresh_delta.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if cv2.contourArea(contour)<1000:
            continue
        status=1
        (x,y,w,h)=cv2.boundingRect(contour)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
    status_list.append(status)
    if status_list[-1]==1 and status_list[-2]==0:
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now())
    
    cv2.imshow("Delta Video",delta_frame)
    cv2.imshow("Threshold Video",thresh_dilate)
    cv2.imshow("Color Frame:",frame)
    
    key=cv2.waitKey(1)
    
    if key==ord('q'):
        if status==1:
            times.append(datetime.now())
        break
print(times)
video.release()
for i in range(0,len(times),2):
    df=df.append({"Start":times[i],"End":times[i+1]},ignore_index=True)
df.to_csv("Times.csv")
cv2.destroyAllWindows()

[datetime.datetime(2019, 3, 26, 22, 26, 49, 255235), datetime.datetime(2019, 3, 26, 22, 26, 52, 196409)]
